<a href="https://colab.research.google.com/github/JO-HEEJIN/AI/blob/master/DeepSeek%2BT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers datasets evaluate rouge_score scikit-learn nltk safetensors

In [2]:
# !pip install bitsandbytes>=0.39.0
# !pip install --upgrade accelerate transformers

In [3]:
# # GPU 메모리 초기화
# cuda.empty_cache()
# torch.backends.cuda.matmul.allow_tf32 = True

NameError: name 'cuda' is not defined

In [6]:
!git clone https://github.com/deepseek-ai/DeepSeek-VL.git
%cd DeepSeek-VL
!pip install -e .

fatal: destination path 'DeepSeek-VL' already exists and is not an empty directory.
/content/DeepSeek-VL
Obtaining file:///content/DeepSeek-VL
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for deepseek_vl (pyproject.toml) ... done
  Created wheel for deepseek_vl: filename=deepseek_vl-1.0.0-0.editable-py3-none-any.whl size=13124 sha256=3c6c9e00d6a1bd894f9873b974a8419a0f583365d89356576fcedc2bb8c5aa43
  Stored in directory: /tmp/pip-ephem-wheel-cache-2afooydf/wheels/44/31/43/66fa81af30d05383b2e1099038fdf053952ce9b0eeef29f928
Successfully built deepseek_vl
  Attempting uninstall: deepseek_vl
    Found existing installation: deepseek_vl 1.0.0
    Uninstalling deepseek_vl-1.0.0:
      Successfully uninstalled deepseek_vl-1.0.0


In [4]:

# 1. GPU 설정 및 메모리 최적화
import torch
from torch import cuda
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    BitsAndBytesConfig,
    AutoProcessor,
    AutoModelForVision2Seq
)
from datasets import load_dataset
import evaluate
import numpy as np
from PIL import Image


In [5]:

# 3. T5 모델 초기화 (FP16 if GPU is available)
def load_t5_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 토크나이저 로드
    tokenizer = T5TokenizerFast.from_pretrained("google/t5-v1_1-small")

    # 모델 로드 (FP16 지원)
    model = T5ForConditionalGeneration.from_pretrained(
        "google/t5-v1_1-small",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,  # GPU 사용 시 FP16
        device_map="auto"
    )

    return tokenizer, model

# ✅ 테스트 실행
tokenizer, model = load_t5_model()
print("🎯 Model & Tokenizer Loaded Successfully!")
print(f"Model Device: {model.device}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


🎯 Model & Tokenizer Loaded Successfully!
Model Device: cuda:0


In [16]:
import torch
from transformers import AutoModelForCausalLM, T5ForConditionalGeneration, T5TokenizerFast
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images
from PIL import Image
import os

# ✅ 1. DeepSeek-VL 모델 로드 (이미지 → 캡션 생성)
def load_deepseek_vl():
    model_path = "deepseek-ai/deepseek-vl-1.3b-chat"

    vl_chat_processor = VLChatProcessor.from_pretrained(model_path)
    tokenizer = vl_chat_processor.tokenizer

    vl_gpt = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
    vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

    return vl_chat_processor, vl_gpt, tokenizer

# # ✅ 2. T5 모델 로드 (캡션 → 키워드 추출)
# def load_t5_model():
#     model_name = "google/t5-v1_1-small"
#     tokenizer = T5TokenizerFast.from_pretrained(model_name)
#     model = T5ForConditionalGeneration.from_pretrained(model_name).cuda().eval()
#     return tokenizer, model

# ✅ 3. 이미지에서 캡션 생성
def generate_caption(image_path, vl_chat_processor, vl_gpt, tokenizer):
    conversation = [
        {"role": "User", "content": "<image_placeholder>Describe this image in detail.", "images": [image_path]},
        {"role": "Assistant", "content": ""}
    ]

    pil_images = load_pil_images(conversation)
    prepare_inputs = vl_chat_processor(conversations=conversation, images=pil_images, force_batchify=True).to(vl_gpt.device)

    inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)
    outputs = vl_gpt.language_model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=prepare_inputs.attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=False,
        use_cache=True
    )

    caption = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
    return caption

# ✅ 4. 캡션에서 키워드 추출
def extract_keywords(caption, t5_tokenizer, t5_model):
    input_text = f"extract keywords: {caption}"
    inputs = t5_tokenizer(input_text, return_tensors="pt").to(t5_model.device)

    outputs = t5_model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    keywords = t5_tokenizer.decode(outputs[0], skip_special_tokens=True).replace("keywords:", "").strip()

    return keywords

# ✅ 5. 전체 워크플로우 실행
def process_image(image_path):
    print(f"📌 Processing Image: {image_path}")

    # 모델 로드
    vl_chat_processor, vl_gpt, vl_tokenizer = load_deepseek_vl()
    t5_tokenizer, t5_model = load_t5_model()

    # 이미지 캡션 생성
    caption = generate_caption(image_path, vl_chat_processor, vl_gpt, vl_tokenizer)
    print(f"📝 Generated Caption: {caption}")

    # 키워드 추출
    keywords = extract_keywords(caption, t5_tokenizer, t5_model)
    print(f"🔑 Extracted Keywords: {keywords}")

    return caption, keywords

# # ✅ 6. Colab 파일 업로드 기능 (테스트용)
# from google.colab import files

# uploaded = files.upload()
# for filename in uploaded.keys():
#     caption, keywords = process_image(filename)


Saving Screenshot 2025-02-13 at 8.30.47 AM.png to Screenshot 2025-02-13 at 8.30.47 AM.png
📌 Processing Image: Screenshot 2025-02-13 at 8.30.47 AM.png


Some kwargs in processor config are unused and will not have any effect: mask_prompt, sft_format, image_tag, ignore_id, add_special_token, num_image_tokens. 


📝 Generated Caption: The image captures a close-up of a tiger's face, set against a blurred background that suggests a natural, outdoor setting. The tiger's eyes, a striking shade of green, are wide open, gazing directly into the camera, creating a sense of connection between the viewer and the subject. The tiger's fur is a mix of brown and black stripes, a characteristic feature of this majestic creature. The whiskers, a defining feature of the tiger, are clearly visible, adding to the overall detail of the image. The image does not contain any text or other discernible objects. The tiger's position in the frame and the direct gaze into the camera give the image a dynamic and engaging quality.
🔑 Extracted Keywords: .com:.com extract .com extract .com extract .com extract .com extract .com.


In [32]:
import torch
from transformers import (
    AutoModelForCausalLM,
    T5ForConditionalGeneration,
    T5TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
from datasets import load_dataset
import evaluate
import numpy as np
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images
from PIL import Image
import os
import ast # 안전한 문자열 → 리스트 변환

# ✅ 1. DeepSeek-VL 모델 로드 (이미지 → 캡션 생성)
def load_deepseek_vl():
    model_path = "deepseek-ai/deepseek-vl-1.3b-chat"

    vl_chat_processor = VLChatProcessor.from_pretrained(model_path)
    tokenizer = vl_chat_processor.tokenizer

    vl_gpt = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
    vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

    return vl_chat_processor, vl_gpt, tokenizer

In [40]:
import torch
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
from datasets import load_dataset
import evaluate
import numpy as np
import os
import ast

# ✅ 1. 안전한 디코딩 함수
def safe_decode(tokenizer, token_ids):
    valid_token_ids = [t for t in token_ids if 0 <= t < tokenizer.vocab_size]
    if not valid_token_ids:
        return ""
    return tokenizer.decode(valid_token_ids, skip_special_tokens=True)

# ✅ 2. T5 모델 로드
def load_t5_model(fine_tuned_path="./fine_tuned_t5"):
    model_name = "google/t5-v1_1-small"
    tokenizer = T5TokenizerFast.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    if os.path.exists(fine_tuned_path):
        print("✅ Loading Fine-Tuned T5 Model")
        model = T5ForConditionalGeneration.from_pretrained(fine_tuned_path).cuda().eval()
    else:
        print("⚠️ Fine-Tuned Model Not Found! Using Pretrained Model Instead.")
        model = T5ForConditionalGeneration.from_pretrained(model_name).cuda().eval()

    return tokenizer, model

# ✅ 3. 데이터셋 로드 및 전처리
def preprocess_function(examples, tokenizer):
    inputs = ["extract keywords: " + caption for caption in examples["caption"]]
    targets = ["keywords: " + ", ".join(ast.literal_eval(kws)) for kws in examples["keywords"]]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        targets,
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# ✅ 4. ROUGE 평가 함수
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = [safe_decode(tokenizer, pred) for pred in predictions]
    decoded_labels = [safe_decode(tokenizer, label) for label in labels]

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    return {k: round(v, 4) for k, v in result.items()}

# ✅ 5. T5 모델 Fine-Tuning
def fine_tune_t5():
    tokenizer = T5TokenizerFast.from_pretrained("google/t5-v1_1-small")
    model = T5ForConditionalGeneration.from_pretrained("google/t5-v1_1-small")

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    model.config.use_cache = False  # 🔹 FP16 문제 해결

    dataset = load_dataset("sachithra913/keywordExtraction")
    tokenized_dataset = dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True, remove_columns=["caption", "keywords"])

    train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(7000))
    eval_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(7000, 9000))
    test_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(9000, 10000))

    # ✅ 학습 설정 (max_new_tokens 삭제)
    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=5,
        predict_with_generate=True,
        generation_max_length=128,  # ✅ max_new_tokens 대신 사용
        bf16=True,  # ✅ FP16 문제 해결: FP16 대신 BF16 사용
        gradient_checkpointing=True,
        optim="paged_adamw_8bit",
        report_to="none",
        logging_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="rougeL"
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    trainer.train()
    trainer.save_model("./fine_tuned_t5")
    print("✅ Fine-Tuning 완료! 🎉")

    # ✅ 평가 수행 (여기서 max_new_tokens 적용)
    predictions = trainer.predict(eval_dataset, max_new_tokens=128)
    print("🔎 Prediction Result:", predictions)

# 실행
fine_tune_t5()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-40-363d3a9fb2b6>:119: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.872400,0.332704,0.141500,0.017800,0.141400,0.141700
2,0.217600,0.073464,0.817600,0.651500,0.809400,0.809300
3,0.118300,0.044563,0.919200,0.835000,0.912000,0.912000
4,0.099600,0.036527,0.945500,0.874600,0.938100,0.938200
5,0.092200,0.035292,0.951200,0.884600,0.943900,0.943900


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


✅ Fine-Tuning 완료! 🎉


🔎 Prediction Result: PredictionOutput(predictions=array([[    0, 12545,    10, ...,  -100,  -100,  -100],
       [    0, 12545,    10, ...,  -100,  -100,  -100],
       [    0, 12545,    10, ...,  -100,  -100,  -100],
       ...,
       [    0, 12545,    10, ...,  -100,  -100,  -100],
       [    0, 12545,    10, ...,  -100,  -100,  -100],
       [    0, 12545,    10, ...,  -100,  -100,  -100]]), label_ids=array([[12545,    10,  7178, ...,     0,     0,     0],
       [12545,    10,   385, ...,     0,     0,     0],
       [12545,    10,   388, ...,     0,     0,     0],
       ...,
       [12545,    10,  3202, ...,     0,     0,     0],
       [12545,    10,  2182, ...,     0,     0,     0],
       [12545,    10,   388, ...,     0,     0,     0]]), metrics={'test_loss': 0.03529224172234535, 'test_rouge1': 0.9512, 'test_rouge2': 0.8846, 'test_rougeL': 0.9439, 'test_rougeLsum': 0.9439, 'test_runtime': 43.0226, 'test_samples_per_second': 46.487, 'test_steps_per_second': 1.464})


In [44]:
def generate_caption(image_path, vl_chat_processor, vl_gpt, tokenizer):
    conversation = [
        {"role": "User", "content": "<image_placeholder>Describe this image in detail.", "images": [image_path]},
        {"role": "Assistant", "content": ""}
    ]

    pil_images = load_pil_images(conversation)
    prepare_inputs = vl_chat_processor(conversations=conversation, images=pil_images, force_batchify=True).to(vl_gpt.device)

    inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)
    outputs = vl_gpt.language_model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=prepare_inputs.attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=False,
        use_cache=True
    )

    caption = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
    return caption

# ✅ 4. 캡션에서 키워드 추출
def extract_keywords(caption, t5_tokenizer, t5_model):
    input_text = f"extract keywords: {caption}"
    inputs = t5_tokenizer(input_text, return_tensors="pt").to(t5_model.device)

    outputs = t5_model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    keywords = t5_tokenizer.decode(outputs[0], skip_special_tokens=True).replace("keywords:", "").strip()

    return keywords

# ✅ 5. 전체 워크플로우 실행
def process_image(image_path):
    print(f"📌 Processing Image: {image_path}")

    # 모델 로드
    vl_chat_processor, vl_gpt, vl_tokenizer = load_deepseek_vl()
    t5_tokenizer, t5_model = load_t5_model()

    # 이미지 캡션 생성
    caption = generate_caption(image_path, vl_chat_processor, vl_gpt, vl_tokenizer)
    print(f"📝 Generated Caption: {caption}")

    # 키워드 추출
    keywords = extract_keywords(caption, t5_tokenizer, t5_model)
    print(f"🔑 Extracted Keywords: {keywords}")

    return caption, keywords

# ✅ 6. Colab 파일 업로드 기능 (테스트용)
from google.colab import files

uploaded = files.upload()
for filename in uploaded.keys():
    caption, keywords = process_image(filename)


Saving Screenshot 2025-02-13 at 8.30.47 AM.png to Screenshot 2025-02-13 at 8.30.47 AM (1).png
📌 Processing Image: Screenshot 2025-02-13 at 8.30.47 AM (1).png


Some kwargs in processor config are unused and will not have any effect: mask_prompt, sft_format, image_tag, ignore_id, add_special_token, num_image_tokens. 


✅ Loading Fine-Tuned T5 Model
📝 Generated Caption: The image captures a close-up of a tiger's face, set against a blurred background that suggests a natural, outdoor setting. The tiger's eyes, a striking shade of green, are wide open, gazing directly into the camera, creating a sense of connection between the viewer and the subject. The tiger's fur is a mix of brown and black stripes, a characteristic feature of this majestic creature. The whiskers, a defining feature of the tiger, are clearly visible, adding to the overall detail of the image. The image does not contain any text or other discernible objects. The tiger's position in the frame and the direct gaze into the camera give the image a dynamic and engaging quality.
🔑 Extracted Keywords: tiger, eyes, wide, open
